# Library

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import zipfile
import glob

# Path & URL Setting

In [3]:
# Define the Target URL of SSKI and folder to save files
target_url = "https://www.bi.go.id/id/statistik/ekonomi-keuangan/seki/Pages/SEKI-JUNI-2025.aspx"

# Relative to the script's location, not the terminal's working directory
# Use the current working directory instead
base_path = os.getcwd()
download_dir = os.path.abspath(os.path.join(base_path, "..", "..", "A. Data Source", "A.1. SEKI (Bank Indonesia)"))

# Make sure the folder exists
os.makedirs(download_dir, exist_ok=True)

# Main Code

In [8]:
# ------------------------------------------------------------------
# 1) WAIT until Chrome finishes all downloads (no *.crdownload files)
# ------------------------------------------------------------------
def wait_downloads(folder, timeout=120):
    """Block until Chrome finishes downloads or timeout (sec)."""
    start = time.time()
    while True:
        tmp_files = [f for f in os.listdir(folder) if f.endswith(".crdownload")]
        if not tmp_files:
            break
        if time.time() - start > timeout:
            raise TimeoutError("Download did not finish in time.")
        time.sleep(1)

## SEKI Downloader from BI Web

In [4]:
month_list = ["JUNI", "DESEMBER"]
year_list = ["2022", "2023", "2024", "2025"]

In [ ]:
# === SETUP SELENIUM ===
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True, 
    "safebrowsing.enabled": True
})
# options.add_argument("--headless")  # optional: run without opening a browser window

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    for year in year_list :
        for month in month_list :
            if year == "2025" and month == "DESEMBER" :
                continue
            else :
                target_url = f"https://www.bi.go.id/id/statistik/ekonomi-keuangan/seki/Pages/SEKI-{month}-{year}.aspx"
            
                # Go to the target URL
                driver.get(target_url)
                time.sleep(5)  # Let the page fully load

                # Find the link in the "Lampiran" section
                download_link = driver.find_element(By.PARTIAL_LINK_TEXT, f"SEKI {month} {year}.zip")
                href = download_link.get_attribute("href")

                # Click the download link
                download_link.click()

                print(f"File is being downloaded from: {href}")
                print(f"Saved to folder: {download_dir}")

                # Wait for download to finish (basic method)
                time.sleep(20)
                wait_downloads(download_dir)   # ← call after download_link.click()

finally:
    driver.quit()

File is being downloaded from: https://www.bi.go.id/id/statistik/ekonomi-keuangan/seki/Documents/SEKI_JUNI_2022.zip
Saved to folder: c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)
File is being downloaded from: https://www.bi.go.id/id/statistik/ekonomi-keuangan/seki/Documents/SEKI_DESEMBER_2022.zip
Saved to folder: c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)
File is being downloaded from: https://www.bi.go.id/id/statistik/ekonomi-keuangan/seki/Documents/SEKI_JUNI_2023.zip
Saved to folder: c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)
File is being downloaded from: https://www.bi.go.id/id/statistik/ekonomi-keuangan/seki/Documents/SEKI_DESEMBER_2023.zip
Saved to folder: c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Ind

## SEKI Downloaded File Unzipper

In [9]:
# ------------------------------------------------------------------
# 2) EXTRACT every .zip found in download_dir
#    Each zip is unpacked into a folder with the same base-name
# ------------------------------------------------------------------
for zip_path in glob.glob(os.path.join(download_dir, "*.zip")):
    base_name = os.path.splitext(os.path.basename(zip_path))[0]
    extract_to = os.path.join(download_dir, base_name)
    os.makedirs(extract_to, exist_ok=True)

    print(f"Extracting {zip_path} → {extract_to}")
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(extract_to)

    # Optional: remove the original zip to save space
    # os.remove(zip_path)

print("✅ All ZIP files extracted.")

Extracting c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)\SEKI_DESEMBER_2022.zip → c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)\SEKI_DESEMBER_2022
Extracting c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)\SEKI_DESEMBER_2023.zip → c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)\SEKI_DESEMBER_2023
Extracting c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)\SEKI_DESEMBER_2024.zip → c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)\SEKI_DESEMBER_2024
Extracting c:\Users\Sharing Vision\Documents\GitHub\Indonesia-Macro-Economic-Analysis\A. Data Source\A.1. SEKI (Bank Indonesia)\SEKI_